<a href="https://colab.research.google.com/github/dlasdowsky/Python/blob/main/Exerc%C3%ADcio_final_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício avaliativo de Python Basic


In [ ]:
!pip install ping3

1. Crie uma classe enviarMensagens que tenha como variável padrão para inicialização a mensagem (Deve estar no __init__)


*   a classe deve ter um método enviar email com parâmetros email e titulo e estes serão usados na biblioteca de envio de email
*   Use a biblioteca smtplib





In [10]:
#Criar aqui a classe
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

class enviarMensagens:
  def __init__ (self, hostSMTP, port, myemail, password = None):
    self.hostSMTP = hostSMTP
    self.port = port
    self.myemail = myemail
    self.password = password
    self.smtp = smtplib.SMTP(host=hostSMTP, port = port)
    self.smtp.starttls()

    if password != None and password != "":
      self.smtp.login(myemail,password)

  def enviarEmail(Self, to, subject, body_message):
    try:
      message = MIMEMultipart("alternative")
      message["Subject"] = subject
      message["From"]= self.myemail
      message.attash(MIMEText(body_message, "plain"))
      self.smtp.sendmail(message["From"], message["To"],message.as_string())

      print("Email enviado com sucesso")
    except Exception as ex:
      print("Error: ", ex)

e = enviarMensagens("smtp.gmail.com", "25", "dlasdowsky@gmail.com")
e.enviarEmail("danieladdpaula@gail.com", "teste titulo","teste msg")

Error:  name 'self' is not defined


2. Crie uma classe com os seguintes métodos:
 

*   verificar_servidor (Use o socket) como parâmetro, o método usa o ip/dominio e a porta, deve salvar em um arquivo csv dia e hora e o resultado ( se conseguiu ou não), por fim, deve enviar email caso tenha dado errado.
*   verificar_portas (Use o nmap) como parâmetro, insira o dominio para teste e o resultado deve ser salvo em um arquivo csv para analises
*   realizar_ping (use uma lib de ping de sua preferência) como parâmetro entre o ip/domínio e salve o resultado em um arquivo csv, caso tenha erro no ping, envie email informando o erro, o dia e hora do ocorrido.


---


**ps.: os dados não podem ser apagados e sim inseridos ao final do arquivo**





In [12]:
# Criar aqui a classe

from socket import *
from ping3 import ping, verbose_ping
import time
import pandas as pd

class Servidor:
  qtd = 0
  def __init__(self, nome, ip):
    self.nome_servidor = nome
    self.ip_servidor = ip
    Servidor.qtd +=1
  def get_ip(self):
    return self.ip_servidor
  def realizar_ping(self):
    return ping(self.ip_servidor)
  @staticmethod
  def quantidade_servidores():
    return Servidor.qtd
import csv
import smtplib
import socket
import subprocess
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

class VerificadorDeServidor:
    
    def __init__(self, email_rem, email_dest):
        self.email_rem = email_rem
        self.email_dest = email_dest
    
    def verificar_servidor(self, host, port):
        # Verifica se o servidor está respondendo na porta especificada
        try:
            socket.setdefaulttimeout(1)
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.connect((host, port))
            s.shutdown(socket.SHUT_RDWR)
            return True
        except:
            return False
        finally:
            s.close()
    
    def verificar_portas(self, host):
        # Verifica as portas abertas no servidor utilizando o nmap
        resultado = subprocess.run(["nmap", "-F", host], capture_output=True, text=True)
        linhas = resultado.stdout.strip().split('\n')
        portas = []
        for linha in linhas:
            if "/tcp" in linha:
                partes = linha.strip().split()
                porta = partes[0].split('/')[0]
                estado = partes[1]
                portas.append((porta, estado))
        return portas
    
    def realizar_ping(self, host):
        # Realiza o ping para o host
        resultado = subprocess.run(["ping", "-c", "1", host], capture_output=True, text=True)
        linhas = resultado.stdout.strip().split('\n')
        for linha in linhas:
            if "packet loss" in linha:
                partes = linha.strip().split(',')
                perda = partes[2].strip().split()[0]
                return perda
        return "Desconhecido"
    
    def salvar_resultado(self, nome_arquivo, dados):
        # Salva os dados em um arquivo CSV
        with open(nome_arquivo, mode='a', newline='') as arquivo:
            writer = csv.writer(arquivo)
            writer.writerow(dados)
    
    def enviar_email(self, assunto, corpo):
        # Envia um email informando um problema com o servidor
        mensagem = MIMEMultipart()
        mensagem['From'] = self.email_rem
        mensagem['To'] = self.email_dest
        mensagem['Subject'] = assunto
        mensagem.attach(MIMEText(corpo))
        with smtplib.SMTP('localhost') as smtp:
            smtp.send_message(mensagem)
    
    def verificar_e_salvar_servidor(self, host, port, nome_arquivo):
        # Verifica o servidor e salva o resultado no arquivo CSV
        resultado = self.verificar_servidor(host, port)
        agora = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        self.salvar_resultado(nome_arquivo, [agora, resultado])
        if not resultado:
            assunto = f"Problema no servidor {host}:{port}"
            corpo = f"O servidor {host}:{port} não está respondendo."
            self.enviar_email(assunto, corpo)
    
    def verificar_e_salvar_portas(self, host, nome_arquivo):
        # Verifica as portas do servidor e salva o resultado no arquivo CSV
        portas = self.verificar_portas(host)
        agora = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        for porta, estado in portas:
            self.salvar_resultado(nome_arquivo, [agora, porta,

import csv
import smtplib
import socket
import subprocess
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

class Servidor:
    def __init__(self, email_from, email_to, email_password):
        self.email_from = email_from
        self.email_to = email_to
        self.email_password = email_password

    def verificar_servidor(self, host, port):
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.settimeout(5)
                s.connect((host, port))
            self._log_result("verificar_servidor", True)
        except (socket.timeout, ConnectionRefusedError):
            self._log_result("verificar_servidor", False)
            self._send_email("Falha na verificação do servidor", f"Não foi possível conectar no host {host}:{port}")

    def verificar_portas(self, host):
        try:
            output = subprocess.check_output(["nmap", "-F", host])
            output_str = output.decode("utf-8")
            open_ports = [line.split("/")[0] for line in output_str.split("\n") if "/tcp" in line and "open" in line]
            self._log_result("verificar_portas", open_ports)
        except subprocess.CalledProcessError:
            self._log_result("verificar_portas", [])
            self._send_email("Falha na verificação de portas", f"Não foi possível verificar as portas do host {host}")

    def realizar_ping(self, host):
        try:
            output = subprocess.check_output(["ping", "-c", "1", host])
            output_str = output.decode("utf-8")
            success = "1 packets transmitted, 1 received" in output_str
            self._log_result("realizar_ping", success)
        except subprocess.CalledProcessError:
            self._log_result("realizar_ping", False)
            self._send_email("Falha no ping", f"Não foi possível realizar o ping no host {host}")

    def _log_result(self, method_name, result):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with open("results.csv", "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([timestamp, method_name, result])

    def _send_email(self, subject, message):
        msg = MIMEMultipart()
        msg['From'] = self.email_from
        msg['To'] = self.email_to
        msg['Subject'] = subject
        msg.attach(MIMEText(message, 'plain'))
        try:
            with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
                smtp.starttls()
                smtp.login(self.email_from, self.email_password)
                smtp.send_message(msg)
        except:
            print("Não foi possível enviar email")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Rode o programa, verificando o servidor, as portas abertas e vendo o ping com o seguintes IP´s/DOminos
dominios_ips = ['186.202.153.153', 'www.google.com.br', 'minhacasa.edu.br']

In [ ]:
# Rode aqui seu código